In [2]:
# STEP 0: Setup & Spark
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "../../")))

from config import settings
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim

spark = SparkSession.builder \
    .appName("Synthetic_ETL_Pipeline") \
    .config("spark.jars", settings.JDBC_PATH) \
    .getOrCreate()

print("✅ Spark session started.")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/20 10:18:47 WARN Utils: Your hostname, Nagendras-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.17 instead (on interface en0)
25/06/20 10:18:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/06/20 10:18:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/Users/aryan/Desktop/project/venv/lib/python3.9/site-packages/google/rpc/__init__.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin t

✅ Spark session started.


In [3]:
import os
import sys
from pyspark.sql import SparkSession

# Set manually for notebooks (since __file__ doesn't exist)
BASE_DIR = os.path.abspath("../../")

sys.path.append(BASE_DIR)
from config.spark_config import get_spark_session

spark = get_spark_session("Read_Parquet_Customers")

parquet_path = os.path.join(BASE_DIR, "data", "parquet_data", "customers_transformed.parquet")

if not os.path.exists(parquet_path):
    raise FileNotFoundError(f"❌ Parquet file not found at: {parquet_path}")
else:
    print(f"✅ Reading Parquet file from: {parquet_path}")

df = spark.read.parquet(parquet_path)

df.printSchema()
df.show(truncate=False)

25/06/20 10:19:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


✅ Reading Parquet file from: /Users/aryan/Desktop/project/data/parquet_data/customers_transformed.parquet
root
 |-- customer_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- company: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone1: string (nullable = true)
 |-- phone2: string (nullable = true)
 |-- email: string (nullable = true)
 |-- subscription_date: date (nullable = true)
 |-- website: string (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email_lower: string (nullable = true)
 |-- country_upper: string (nullable = true)
 |-- name_length: integer (nullable = true)
 |-- ingested_at: timestamp (nullable = true)

+---------------+----------+-----------+----------------------------+----------------+---------------------------------------------------+---------------------+----------------------+-----------------------------+-----------------

In [4]:
from pyspark.sql.functions import upper

# Filter rows where country_upper == 'INDIA'
df_india = df.filter(upper(df["country_upper"]) == "INDIA")

# Show result
df_india.show(truncate=False)
df_india.count()

+---------------+----------+---------+------------------------------+-------------------+-------+----------------------+----------------------+-----------------------------------+-----------------+-------------------------------+-----------------+-----------------------------------+-------------+-----------+--------------------------+
|customer_id    |first_name|last_name|company                       |city               |country|phone1                |phone2                |email                              |subscription_date|website                        |full_name        |email_lower                        |country_upper|name_length|ingested_at               |
+---------------+----------+---------+------------------------------+-------------------+-------+----------------------+----------------------+-----------------------------------+-----------------+-------------------------------+-----------------+-----------------------------------+-------------+-----------+-----------------

8279

In [ ]:
from pyspark.sql import SparkSession
from datetime import datetime
import sys
import os
from config import settings 
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
from pyspark.sql import SparkSession
# Initialize Spark session
spark = SparkSession.builder \
    .appName("ReadFromPostgresAndWriteParquet") \
    .config("spark.jars", settings.JDBC_PATH) \
    .getOrCreate()
props = {
    "user": settings.POSTGRES_USER,
    "password": settings.POSTGRES_PASSWORD,
    "driver": "org.postgresql.Driver"
}
df = spark.read.jdbc(
    url=settings.JDBC_URL,
    table="customers_2",
    properties=props
)
df.show()





25/06/20 10:34:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    customer_id|first_name|last_name|             company|             city|             country|              phone1|              phone2|               email|subscription_date|             website|
+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|4962fdbE6Bfee6D|       Pam|   Sparks|        Patel-Deleon|       Blakemouth|British Indian Oc...|    267-243-9490x035|    480-078-0535x889|nicolas00@faulkne...|       2020-11-29| https://nelson.com/|
|9b12Ae76fdBc9bE|      Gina|    Rocha|Acosta, Paul and ...| East Lynnchester|          Costa Rica|        027.142.0940|+1-752-593-4777x0...|  yfarley@morgan.com|       2021-01-03|https://pineda-ro